In [1]:
import numpy as np
import pandas as pd

import string
import re

In [2]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [3]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import CountVectorizer, IDF
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType, BooleanType
from pyspark.ml import Pipeline

In [4]:
from nlp_cl_start import data_tokenizer, if_rest_udf
from pipe_spar import api_f, cluster_biz_by_review , cluster_user_by_review
from my_metr import transform_to_score, my_scorer, transform_aggregated
from sklearn.metrics import accuracy_score, recall_score

In [5]:
import pyspark as ps
spark = (ps.sql.SparkSession.builder
        .master("local[4]")
        .appName("yelp_academic")
        .getOrCreate()
        )
sc = spark.sparkContext
seed = 91

In [6]:
%%time
from search_param import create_test_set

CPU times: user 2.71 ms, sys: 1.69 ms, total: 4.4 ms
Wall time: 5.74 ms


In [7]:
biz = spark.read.json('yelp_dataset/yelp_academic_dataset_business.json')
rev = spark.read.json('yelp_dataset/yelp_academic_dataset_review.json')

rests = biz.filter(if_rest_udf(biz.categories))

rest_rev = rev.join(rests.select('business_id','stars').withColumnRenamed('stars','rating'),'business_id')
bad_reviews = rest_rev.filter('stars < 3')

bad_sample = bad_reviews.sample(False, 0.127, seed =seed)

bad_sample.cache()
sample_token= data_tokenizer(bad_sample)

splits = sample_token.randomSplit([0.8, 0.1, 0.1], seed = seed)
train = splits[0]
add_cl = splits[1]
test = splits[2]


In [8]:
train.first()

Row(business_id='--9e1ONYQuAa-CB_Rrw7Tw', cool=0, date='2012-05-11', funny=0, review_id='OaRMJKI6S7LAoa8xoEaqPg', stars=2, text="I feel like Twitter is pretty useless so far; I have an ok amount of followers I guess, but I feel like they're all following so many people that anything I say gets lost in the mess of their feed. I could tweet all day and get very little response to anything aside from a couple loyal folks on there. And it was one such reader that led me to Delmonico recently for what he claimed to be an excellent burger. He's a food writer in town so he should know what he's talking about, right?\n\nDelmonico is located in the small restaurant row of the Venetian right by another great burger spot, although most people would not come here for their burger. The interior is nothing too fancy in either the bar area or more traditional area in the back. I did notice while sitting at the bar that they have an amazing bourbon selection, but since I was there during a lunch break

In [9]:
params_kcv = {'minDF' : 10, 'vocabSize':5000, 'k':15}

cv = CountVectorizer(minDF=params_kcv['minDF'],
        vocabSize=params_kcv['vocabSize'], inputCol='token', outputCol='vectors')
km1 = KMeans(k = params_kcv['vocabSize'], featuresCol='vectors', maxIter= 30)
pipe_count = Pipeline(stages=[cv, km1])

In [10]:
params_kidf = {'minDF' : 10, 'vocabSize':5000, 'k':15, 'minDocFreq' : 2}

cv = CountVectorizer(minDF=params_kidf['minDF'],
        vocabSize=params_kidf['vocabSize'], inputCol='token', outputCol='vectors')

idf = IDF(minDocFreq= params_kidf['minDocFreq'], inputCol="vector", outputCol="features")
km2 = KMeans(k = params_kidf['k'], featuresCol='features', maxIter= 30)
pipe_idf = Pipeline(stages = [cv, idf, km2])

In [11]:
%%time
both = train.union(add_cl)

CPU times: user 527 µs, sys: 1.25 ms, total: 1.78 ms
Wall time: 30.4 ms


In [17]:
cvm = cv.fit(train)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:51433)
Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.3.2/libexec/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.3.2/libexec/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 61] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:51433)

In [14]:
%%time
new_grf = create_test_set(both, count_model, rest_rev)

NameError: name 'count_model' is not defined

In [13]:
results = my_scorer(new_grf)

NameError: name 'new_grf' is not defined